In [0]:
from keras.layers import Input, Dense, Conv2D
from keras.models import Model
import numpy as np
import tensorflow as tf
import cv2, sys
import imageio as io
from matplotlib import pyplot as plt

In [0]:
def decode_float(fnum):
      
      # split int and dec parts
      fract,integ=np.modf(fnum)
      
      intstr=bin(np.int(integ))[2:]
      floatstr=''

      while fnum!=0.0:
          fnum=fract*2
          fract,integ=np.modf(fnum)
          floatstr=floatstr+str(np.int(integ))
          fnum=fract

      # pad zeros to adjust length
      intstr = (8-len(intstr))*'0' + intstr
      floatstr += (8-len(floatstr))*'0'
      
      return intstr+floatstr

def float2bin(im,stride=16):
    
     # Compute size of extracted output image
     h,w=im.shape[0], im.shape[1]
     result=np.zeros((h,w*stride), dtype=np.uint8)
     
     # 2D convolution with stride of 16 [kernel shape: (1,16)]
     for i in range(0,h):
         for j in range(0,w):
             ridx=j*stride
             bval=decode_float(im[i,j])
             result[i,ridx:ridx+stride]=np.uint8(list(bval))*np.uint8([255])
             #print("BBVAL: ",bval,im[i,j], (i,ridx) )
     return result

In [0]:
# Identity kernel (testing)
def kernel_init1(shape, dtype=None):
    
    kernel = np.zeros(shape)
    kernel[:,:,0,0] = np.array([[0,0,0],[0,1,0],[0,0,0]])
    return kernel

# Fixed point fp16 kernel (compression)
def kernel_init2(shape, dtype=None):
  
    base=np.array([7,6,5,4,3,2,1,0,-1,-2,-3,-4,-5,-6,-7,-8],dtype=np.float32)
    kernel=np.power(2,base).reshape(shape)
    return kernel

In [0]:
# Test model 1 (Identity)
inputs = Input(shape=(256,256,1))
outputs = Conv2D(1, (3, 3), kernel_initializer=kernel_init1, padding="same" )(inputs)
model1 = Model(inputs=inputs, outputs=outputs)

model1.summary()
model1.save('model1.h5')

In [0]:
# Test model 2 (Compression)
inputs = Input(shape=(256,256,1))
hidden = Conv2D(1, (3, 3), kernel_initializer=kernel_init1, padding="same" )(inputs)
outputs = Conv2D(1, (1, 16), kernel_initializer=kernel_init2,strides=(1,16), padding="same" )(hidden)
model2 = Model(inputs=inputs, outputs=outputs)

model2.summary()
model2.save('model2.h5')

In [0]:
# Read input and preprocess
input_img = io.imread('https://drive.google.com/uc?id=1LAfmxWqXVUki71Ha_mWbQ4yrGIOTUGHV')
input_img=input_img[:,:,0]
img=np.float32(input_img/255.0)
img=img.reshape((1,256,256,1))

In [0]:
# Perform prediction
out1=model1.predict(img)
out2=model2.predict(img)

In [0]:
# Plot output of model1
plt.imshow(out1.squeeze())

In [0]:
output_img=float2bin(out2.squeeze()) # Extracted image [UINT8]

print("\nConverting the extracted image back to UINT8...")

print("\nExtracted image:- ")
print("Shape: ",output_img.shape)
print("Type: ", output_img.dtype)
print("Size: ",sys.getsizeof(output_img))
print("Unique values: ", np.unique(output_img).shape[0])

# Check if image was successfully extracted
if np.array_equal(input_img,output_img):
   print("\nExtraction successful !!!")
else:
   print("\nExtraction failed !!!")

# Save the output image
cv2.imwrite("decoded2.png",output_img)
plt.imshow(output_img)

In [0]:
# Export mode1-1 as tflite
converter = tf.lite.TFLiteConverter.from_keras_model_file('/content/model1.h5',custom_objects={'kernel_init1':kernel_init1})
tflite_model1 = converter.convert()
open("model1.tflite", "wb").write(tflite_model1)


In [0]:
# Export model-2 as tflite
converter = tf.lite.TFLiteConverter.from_keras_model_file('/content/model2.h5', custom_objects={'kernel_init1':kernel_init1, 'kernel_init2':kernel_init2})
tflite_model2 = converter.convert()
open("model2.tflite", "wb").write(tflite_model2)